# Make the table that lists the full QPQ9 sample

In [5]:
# imports
from __future__ import print_function,absolute_import,division,unicode_literals
import numpy as np
import glob,os,sys,copy,imp 
from scipy import stats as scistats
import matplotlib as mpl
mpl.rcParams['font.family']='stixgeneral'
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
from astropy.table import QTable,Table
from astropy.io import ascii,fits
from astropy import units as u
from astropy import constants as const
from astropy.coordinates import SkyCoord
from linetools.spectralline import AbsLine
import linetools.utils as ltu
from xastropy.igm.abs_sys import abssys_utils as abssys
from xastropy import spec as xpsec
from xastropy.plotting import utils as xputils
from xastropy.xutils import xdebug as xdb
from xastropy.obs import radec as xor
from xastropy.atomic import ionization as xai
from xastropy.xutils import fits as xxf
sys.path.append(os.path.abspath("../../../py"))
from enigma.qpq import utils as qpqutils
from enigma.qpq import spec as qpqs
from enigma.qpq import stacks as qpqk
sys.path.append(os.path.abspath("../Analysis/Stacks/py"))
import qpq9_stacks as qpq9k

In [6]:
# Load QPQ9 structure
path = enigma.__path__[0]+'/data/qpq/'
QPQ9 = xxf.bintab_to_table(path+'qpq9_final.fits')

x.fits.bintab_to_table: Reading /Users/lwymarie/python/enigma/enigma/data/qpq/qpq9_final.fits


In [208]:
# Load the dictionaries of quasars used in stack for CII
stack_tup0 = qpq9k.qpq9_IRMgII(passback=True,wrest=1334.5323*u.AA,S2N_cut=5.5/u.AA,zfg_mnx=(1.6,9999),
                               plot_indiv=False)
# Mask
fin_velo, stck_img, stck_msk, all_dict = stack_tup0
stck_mskN = copy.deepcopy(stck_msk)
idx_mask = []
for ii,idict in enumerate(all_dict):
    if idict is None:
        continue
    if 'J1508+3635' in idict['qpq']['NAME']: #DLA not excluded by forest cut, should be excluded now
        idx_mask.append(ii)
for idx in idx_mask:
    stck_mskN[idx,:] = 0.

Reading /Users/lwymarie/python/enigma/enigma/data/qpq/qpq9_final.fits
4.5030178527 1 / Angstrom
No spectrum found for BOSSJ0008-0039 and wvobs=3920.97 Angstrom!
3.96814209033 1 / Angstrom
No spectrum found for BOSSJ0042+1350 and wvobs=3704.79 Angstrom!
81.1223225091 1 / Angstrom
44.1513656402 1 / Angstrom
Using /u/xavier/Dropbox//QSOPairs/data/GMOS_redux//./SDSSJ034139.19-000012.7_b1200_F.fits.gz for SDSSJ0341-0000
S/N 81.1223225091 1 / Angstrom
91.8691590217 1 / Angstrom
Using /u/xavier/Dropbox//QSOPairs/data/LRIS_redux//./SDSSJ080048.74+354231.3_b1200_F.fits.gz for APOJ0800+3542
S/N 91.8691590217 1 / Angstrom
6.82627542183 1 / Angstrom
Using spec-7328-56715-0428.fits.gz for BOSSJ0809+4539
S/N 6.82627542183 1 / Angstrom
No spectrum found for BOSSJ0826+4708 and wvobs=3734.97 Angstrom!
7.18768185783 1 / Angstrom
30.4325951329 1 / Angstrom
Using /u/xavier/Dropbox//QSOPairs/data/LRIS_redux//./SDSSJ082844.87+454518.2_b1200_F.fits.gz for SDSSJ0828+4545
S/N 30.4325951329 1 / Angstrom
30.2325

In [209]:
# Save the properties of pairs used in stack for CII
sv_fg_qso = []
sv_zfg = []
sv_line = []
sv_bg_qso = []
sv_zbg = []
sv_bg_instr = []
sv_Rphys = []
sv_gUV = []
for ii,idict in enumerate(all_dict):
    if idict is None:
        continue
    if np.sum(stck_mskN[ii,:]) == 0:
        continue
    # fg qso 
    coord = SkyCoord(idict['qpq']['FG_RA']*u.deg,idict['qpq']['FG_DEC']*u.deg)
    RA_str = coord.ra.to_string(unit=u.hour,sep='',pad=True)
    RA_fmt = RA_str[0:8]+str(np.around(float(RA_str),decimals=2))[-1]
    dec_str = coord.dec.to_string(unit=u.deg,sep='',pad=True,alwayssign=True)
    if '-' in dec_str:
        dec_fmt = '$-$' + dec_str[1:8]+str(np.around(float(dec_str),decimals=1))[-1]
    else:
        dec_fmt = dec_str[0:8]+str(np.around(float(dec_str),decimals=1))[-1]
    coord_fmt = 'J'+RA_fmt+dec_fmt
    sv_fg_qso.append(coord_fmt)
    # z_fg
    zfg = str(np.around(idict['qpq']['Z_FG'],decimals=4))
    sv_zfg.append(zfg)
    # line analyzed for z_fg 
    line = idict['qpq']['ZFG_LINE']
    sv_line.append(line)
    # bg qso 
    coord = SkyCoord(idict['qpq']['RA']*u.deg,idict['qpq']['DEC']*u.deg)
    RA_str = coord.ra.to_string(unit=u.hour,sep='',pad=True)
    RA_fmt = RA_str[0:8]+str(np.around(float(RA_str),decimals=2))[-1]
    dec_str = coord.dec.to_string(unit=u.deg,sep='',pad=True,alwayssign=True)
    dec_fmt = dec_str[0:8]+str(np.around(float(dec_str),decimals=1))[-1]
    coord_fmt = 'J'+RA_fmt+dec_fmt
    sv_bg_qso.append(coord_fmt)
    # z_bg 
    zbg = str(np.around(idict['qpq']['BG_Z'],decimals=3))
    sv_zbg.append(zbg)
    # bg qso instr
    bg_instr = idict['instr']
    if bg_instr == 'LRISb':
        bg_instr = 'LRIS'
    if bg_instr == 'MIKE-Blue':
        bg_instr = 'MIKE'
    if bg_instr == 'MODS1B':
        bg_instr = 'MODS1'
    sv_bg_instr.append(bg_instr)
    # R_perp 
    R_phys = str(np.int(np.rint(idict['qpq']['R_PHYS'])))
    sv_Rphys.append(R_phys)
    # g_UV 
    g_UV = str(np.int(np.rint(idict['qpq']['G_UV'])))
    sv_gUV.append(g_UV)

In [210]:
# Load the dictionaries of quasars used in stack for CIV
stack_tup0 = qpq9k.qpq9_IRMgII(passback=True,wrest=1548.195*u.AA,S2N_cut=5.5/u.AA,zfg_mnx=(1.6,9999),
                              plot_indiv=False)
# Mask
fin_velo, stck_img, stck_msk, all_dict = stack_tup0
stck_mskN = copy.deepcopy(stck_msk)
idx_mask = []
for ii,idict in enumerate(all_dict):
    if idict is None:
        continue
    if 'J1002+0020' in idict['qpq']['NAME']: #CIV overlaps with BAL of background quasar
        idx_mask.append(ii) 
for idx in idx_mask:
    stck_mskN[idx,:] = 0.

Reading /Users/lwymarie/python/enigma/enigma/data/qpq/qpq9_final.fits
4.16329841575 1 / Angstrom
No spectrum found for BOSSJ0008-0039 and wvobs=4045.27 Angstrom!
5.31772010709 1 / Angstrom
No spectrum found for BOSSJ0008-0039 and wvobs=4548.73 Angstrom!
7.92763897204 1 / Angstrom
Using spec-6197-56191-0089.fits.gz for BOSSJ0042+1350
S/N 7.92763897204 1 / Angstrom
5.99371003287 1 / Angstrom
Using /u/xavier/Dropbox//QSOPairs/data/BOSS/v5_5_0/SDSSJ005718.90-000134.7.fits for BOSSJ0057-0001
S/N 5.99371003287 1 / Angstrom
13.151452365 1 / Angstrom
Using spec-3735-55209-0693.fits.gz for BOSSJ0103-0002
S/N 13.151452365 1 / Angstrom
No spectrum found for SDSSJ0111+1402 and wvobs=5395.68 Angstrom!
7.58612977327 1 / Angstrom
Using /u/xavier/Dropbox//QSOPairs/data/SDSS//./SDSSJ014917.46-002158.4.fits.gz for SDSSJ0149-0021
S/N 7.58612977327 1 / Angstrom
10.4266274166 1 / Angstrom
Using spec-4236-55479-0340.fits.gz for BOSSJ0214-0052
S/N 10.4266274166 1 / Angstrom
12.034277869 1 / Angstrom
2.101003

In [211]:
# Save the properties
for ii,idict in enumerate(all_dict):
    if idict is None:
        continue
    if np.sum(stck_mskN[ii,:]) == 0:
        continue
    # fg qso 
    coord = SkyCoord(idict['qpq']['FG_RA']*u.deg,idict['qpq']['FG_DEC']*u.deg)
    RA_str = coord.ra.to_string(unit=u.hour,sep='',pad=True)
    RA_fmt = RA_str[0:8]+str(np.around(float(RA_str),decimals=2))[-1]
    dec_str = coord.dec.to_string(unit=u.deg,sep='',pad=True,alwayssign=True)
    if '-' in dec_str:
        dec_fmt = '$-$' + dec_str[1:8]+str(np.around(float(dec_str),decimals=1))[-1]
    else:
        dec_fmt = dec_str[0:8]+str(np.around(float(dec_str),decimals=1))[-1]
    coord_fmt = 'J'+RA_fmt+dec_fmt
    if coord_fmt in sv_fg_qso: # already saved for CII stack 
        continue
    sv_fg_qso.append(coord_fmt)
    # z_fg
    zfg = str(np.around(idict['qpq']['Z_FG'],decimals=4))
    sv_zfg.append(zfg)
    # line analyzed for z_fg 
    line = idict['qpq']['ZFG_LINE']
    sv_line.append(line)
    # bg qso 
    coord = SkyCoord(idict['qpq']['RA']*u.deg,idict['qpq']['DEC']*u.deg)
    RA_str = coord.ra.to_string(unit=u.hour,sep='',pad=True)
    RA_fmt = RA_str[0:8]+str(np.around(float(RA_str),decimals=2))[-1]
    dec_str = coord.dec.to_string(unit=u.deg,sep='',pad=True,alwayssign=True)
    dec_fmt = dec_str[0:8]+str(np.around(float(dec_str),decimals=1))[-1]
    coord_fmt = 'J'+RA_fmt+dec_fmt
    sv_bg_qso.append(coord_fmt)
    # z_bg 
    zbg = str(np.around(idict['qpq']['BG_Z'],decimals=3))
    sv_zbg.append(zbg)
    # bg qso instr
    bg_instr = idict['instr']
    if bg_instr == 'LRISb':
        bg_instr = 'LRIS'
    if bg_instr == 'MIKE-Blue':
        bg_instr = 'MIKE'
    if bg_instr == 'MODS1B':
        bg_instr = 'MODS1'
    sv_bg_instr.append(bg_instr)
    # R_perp 
    R_phys = str(np.int(np.rint(idict['qpq']['R_PHYS'])))
    sv_Rphys.append(R_phys)
    # g_UV 
    g_UV = str(np.int(np.rint(idict['qpq']['G_UV'])))
    sv_gUV.append(g_UV)

In [212]:
# Load the dictionaries of quasars used in stack for MgII 
stack_tup0 = qpq9k.qpq9_IRMgII(passback=True,wrest=2796.354*u.AA,S2N_cut=5.5/u.AA,zfg_mnx=(1.6,9999),
                              plot_indiv=False)
fin_velo, stck_img, stck_msk, all_dict = stack_tup0
stck_mskN = copy.deepcopy(stck_msk)
idx_mask = []
for ii,idict in enumerate(all_dict):
    if idict is None:
        continue
    if 'J0822+1319' in idict['qpq']['NAME']: #BAL
        idx_mask.append(ii) 
for idx in idx_mask:
    stck_mskN[idx,:] = 0.

Reading /Users/lwymarie/python/enigma/enigma/data/qpq/qpq9_final.fits
1.95950620869 1 / Angstrom
No spectrum found for BOSSJ0001+0000 and wvobs=8359.87 Angstrom!
3.90199952094 1 / Angstrom
No spectrum found for BOSSJ0008-0039 and wvobs=7306.58 Angstrom!
3.24163041775 1 / Angstrom
No spectrum found for BOSSJ0008-0039 and wvobs=8215.93 Angstrom!
5.22357649931 1 / Angstrom
No spectrum found for BOSSJ0042+1350 and wvobs=7762.94 Angstrom!
21.5242916444 1 / Angstrom
15.5022425795 1 / Angstrom
Using spec-6203-56266-0300.fits.gz for BOSSJ0047+1447
S/N 21.5242916444 1 / Angstrom
0.94873000285 1 / Angstrom
No spectrum found for BOSSJ0057-0001 and wvobs=8838.87 Angstrom!
7.12294300403 1 / Angstrom
BOSSJ0103-0002 does not survive atmosphere cut
No spectrum found for BOSSJ0103-0002 and wvobs=7700.44 Angstrom!
5.13291643988 1 / Angstrom
No spectrum found for BOSSJ0106+0056 and wvobs=8020.81 Angstrom!
No spectrum found for SDSSJ0111+1402 and wvobs=9745.69 Angstrom!
7.99245858919 1 / Angstrom
Using /u

In [213]:
# Save the properties
for ii,idict in enumerate(all_dict):
    if idict is None:
        continue
    if np.sum(stck_mskN[ii,:]) == 0:
        continue
    # fg qso 
    coord = SkyCoord(idict['qpq']['FG_RA']*u.deg,idict['qpq']['FG_DEC']*u.deg)
    RA_str = coord.ra.to_string(unit=u.hour,sep='',pad=True)
    RA_fmt = RA_str[0:8]+str(np.around(float(RA_str),decimals=2))[-1]
    dec_str = coord.dec.to_string(unit=u.deg,sep='',pad=True,alwayssign=True)
    if '-' in dec_str:
        dec_fmt = '$-$' + dec_str[1:8]+str(np.around(float(dec_str),decimals=1))[-1]
    else:
        dec_fmt = dec_str[0:8]+str(np.around(float(dec_str),decimals=1))[-1]
    coord_fmt = 'J'+RA_fmt+dec_fmt
    if coord_fmt in sv_fg_qso: # already saved for CII, CIV stack 
        continue
    sv_fg_qso.append(coord_fmt)
    # z_fg
    zfg = str(np.around(idict['qpq']['Z_FG'],decimals=4))
    sv_zfg.append(zfg)
    # line analyzed for z_fg 
    line = idict['qpq']['ZFG_LINE']
    sv_line.append(line)
    # bg qso 
    coord = SkyCoord(idict['qpq']['RA']*u.deg,idict['qpq']['DEC']*u.deg)
    RA_str = coord.ra.to_string(unit=u.hour,sep='',pad=True)
    RA_fmt = RA_str[0:8]+str(np.around(float(RA_str),decimals=2))[-1]
    dec_str = coord.dec.to_string(unit=u.deg,sep='',pad=True,alwayssign=True)
    dec_fmt = dec_str[0:8]+str(np.around(float(dec_str),decimals=1))[-1]
    coord_fmt = 'J'+RA_fmt+dec_fmt
    sv_bg_qso.append(coord_fmt)
    # z_bg 
    zbg = str(np.around(idict['qpq']['BG_Z'],decimals=3))
    sv_zbg.append(zbg)
    # bg qso instr
    bg_instr = idict['instr']
    if bg_instr == 'LRISb':
        bg_instr = 'LRIS'
    if bg_instr == 'MIKE-Blue':
        bg_instr = 'MIKE'
    if bg_instr == 'MODS1B':
        bg_instr = 'MODS1'
    sv_bg_instr.append(bg_instr)
    # R_perp 
    R_phys = str(np.int(np.rint(idict['qpq']['R_PHYS'])))
    sv_Rphys.append(R_phys)
    # g_UV 
    g_UV = str(np.int(np.rint(idict['qpq']['G_UV'])))
    sv_gUV.append(g_UV)

In [214]:
tab = Table([sv_fg_qso,sv_zfg,sv_line,sv_bg_qso,sv_zbg,sv_bg_instr,sv_Rphys,sv_gUV],
            names=['fg_qso','zfg','line','bg_qso','zbg','bg_instr','Rphys','gUV'])
tab.sort('fg_qso')
tab

fg_qso,zfg,line,bg_qso,zbg,bg_instr,Rphys,gUV
unicode21,str6,str9,unicode19,str5,unicode8,str3,str5
J004238.24+135054.8,1.7761,SDSS-HW,J004236.53+135037.3,1.894,BOSS,266,386
J004757.26+144741.0,1.6197,SDSS-HW,J004757.88+144744.7,2.757,BOSS,82,6705
J005717.36$-$000113.3,2.1609,[OIII],J005718.99-000134.7,2.511,BOSS,271,1283
J010323.84$-$000254.2,1.7537,SDSS-HW,J010324.37-000251.3,2.306,BOSS,74,5488
J014917.11$-$002141.6,1.6823,MgII,J014917.46-002158.5,2.159,SDSS,155,2390
J021416.96$-$005229.1,1.8019,SDSS-HW,J021416.12-005251.5,2.332,BOSS,225,566
J022447.89$-$004700.4,1.6959,SDSS-HW,J022448.85-004638.9,2.188,BOSS,226,294
J023018.27$-$033319.4,2.3802,MgII,J023019.99-033315.0,2.985,BOSS,221,1444
J023946.43$-$010640.4,2.2988,[OIII],J023946.45-010644.1,3.124,BOSS,32,20931


In [218]:
# Print to a tex file 
afile = open('tab_sample.tex','w')
afile.write('\\LongTables\n')
afile.write('\\begin{deluxetable*}{ccccccccc}\n')
afile.write('\\tablewidth{0pc}\n')
afile.write('\\tablecaption{Properties of the Projected Quasar Pairs in the QPQ9 sample')
afile.write('\\label{tab:sample}}\n')
afile.write('\\tabletypesize{\\scriptsize}\n')
afile.write('\\setlength{\\tabcolsep}{0in}\n')
afile.write('\\tablehead{\\colhead{Foreground Quasar} & \\colhead{$z_{\\rm fg}$} & \n')
afile.write('\\colhead{Line for $z_{\\rm fg}^a$} & \\colhead{Background Quasar} & \n')
afile.write('\\colhead{$z_{\\rm bg}$} & \\colhead{BG Quasar Instrument} & \n')
afile.write('\\colhead{$R_\\perp$ (kpc)} & \\colhead{$g_{\\rm UV}$}} \n')
afile.write('\\startdata \n')
for ii in np.arange(len(tab)):
    afile.write(tab['fg_qso'][ii] + ' & ' + tab['zfg'][ii] + ' & ' + tab['line'][ii] + '& ')
    afile.write(tab['bg_qso'][ii] + ' & ' + tab['zbg'][ii] + ' & ' + tab['bg_instr'][ii] + ' & ')
    afile.write(tab['Rphys'][ii] + ' & ' + tab['gUV'][ii] + ' \\\ \n')
afile.write('\\enddata \n')
afile.write('\\tablenotetext{a}{The emission-line analyzed for measuring $z_{\\rm fg}$. \n')
afile.write('SDSS-HW stands for adopting the redshift measured by Hewett \\& Wild (2010) \n')
afile.write('directly.} \n')
afile.write('\\end{deluxetable*}')
afile.close()